In [1]:
from aocd import get_data

# Day 5: Print Queue

## Part One

Safety protocols clearly indicate that new pages for the safety manuals must be printed in a very specific order. The notation `X|Y` means that if both page number `X` and page number `Y` are to be produced as part of an update, page number `X` must be printed at some point before page number `Y`.

The Elf has for you both the page ordering rules and the pages to produce in each update (your puzzle input), but can't figure out whether each update has the pages in the right order.

The first section specifies the page ordering rules, one per line. The first rule, `47|53`, means that if an update includes both page number `47` and page number `53`, then page number `47` must be printed at some point before page number `53`. (`47` doesn't necessarily need to be immediately before `53`; other pages are allowed to be between them.)

The second section specifies the page numbers of each update. Because most safety manuals are different, the pages needed in the updates are different too. The first update, `75,47,61,53,29`, means that the update consists of page numbers `75`, `47`, `61`, `53`, and `29`.

Start by identifying which updates are already in the right order.

In the above example, the first update `(75,47,61,53,29)` is in the right order:<br/>
`75` is correctly first because there are rules that put each other page after it: `75|47`, `75|61`, `75|53`, and `75|29`.<br/>
`47` is correctly second because `75` must be before it `(75|47)` and every other page must be after it according to `47|61`, `47|53`, and `47|29`.<br/>
`61` is correctly in the middle because `75` and `47` are before it (`75|61` and `47|61`) and `53` and `29` are after it (`61|53` and `61|29`).<br/>
`53` is correctly fourth because it is before page number `29` (`53|29`).<br/>
`29` is the only page left and so is correctly last.<br/>

Because the first update does not include some page numbers, the ordering rules involving those missing page numbers are ignored.

In [2]:
example_input = """
47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47
"""

In [3]:
import numpy as np

def load_rules_and_updates(data):
    rules_part, updates_part = data.strip().split('\n\n')
    
    rules = np.array([tuple(line.split('|')) for line in rules_part.strip().split('\n')], dtype=int)
    
    updates = [np.array(line.split(','), dtype=int) for line in updates_part.strip().split('\n')]
    
    return np.array(rules), updates

In [4]:
example_rules, example_updates = load_rules_and_updates(example_input); example_rules[:5]

array([[47, 53],
       [97, 13],
       [97, 61],
       [97, 47],
       [75, 29]])

First figure out which rules are relevant to the update

In [5]:
example_updates

[array([75, 47, 61, 53, 29]),
 array([97, 61, 53, 29, 13]),
 array([75, 29, 13]),
 array([75, 97, 47, 61, 53]),
 array([61, 13, 29]),
 array([97, 13, 75, 29, 47])]

In [6]:
matches_left = np.isin(example_rules[:, 0], example_updates[0])
matches_right = np.isin(example_rules[:, 1], example_updates[0])

both_matches = matches_left & matches_right; both_matches

array([ True, False, False, False,  True, False,  True, False, False,
        True,  True, False,  True, False,  True, False,  True,  True,
        True, False, False])

In [7]:
from collections import defaultdict

def build_graph(rules):
    """
    Graph where the key represents the number that must come 
    ahead of any of the elements in the set of value numbers
    """
    graph = defaultdict(set)
    for x, y in rules:
        graph[x].add(y)
    return graph

In [8]:
def is_update_valid(update, rules, rules_graph):
    matches_left = np.isin(rules[:, 0], list(update))
    matches_right = np.isin(rules[:, 1], list(update))
    
    both_matches = matches_left & matches_right
    
    # map array values to indices for efficient lookups
    value_to_index = {value: idx for idx, value in enumerate(update)}
    
    for key in set(rules[both_matches][:,0]):
        key_idx = value_to_index.get(key, None)
        
        for value in rules_graph[key]:
            if value_idx := value_to_index.get(value, None):
                if value_idx <= key_idx:
                    return False
    
    return True

In [9]:
rules_graph = build_graph(example_rules)
for update in example_updates:
    print(f"{update} is{' not' if not is_update_valid(update, example_rules, rules_graph) else ''} valid")

[75 47 61 53 29] is valid
[97 61 53 29 13] is valid
[75 29 13] is valid
[75 97 47 61 53] is valid
[61 13 29] is not valid
[97 13 75 29 47] is not valid


In [10]:
data = get_data(day=5, year=2024)
rules, updates = load_rules_and_updates(data)

In [11]:
sum_valid = 0
rules_graph = build_graph(rules)
for update in updates:
    if is_update_valid(update, rules, rules_graph):
        sum_valid += update[len(update)//2]
sum_valid

np.int64(4609)

## Part Two

For each of the incorrectly-ordered updates, use the page ordering rules to put the page numbers in the right order. For the above example, here are the three incorrectly-ordered updates and their correct orderings:

`75,97,47,61,53` becomes `97,75,47,61,53`.<br/>
`61,13,29` becomes `61,29,13`.<br/>
`97,13,75,29,47` becomes `97,75,47,29,13`.<br/>

After taking only the incorrectly-ordered updates and ordering them correctly, their middle page numbers are `47`, `29`, and `47`. Adding these together produces `123`.

Find the updates which are not in the correct order. What do you get if you add up the middle page numbers after correctly ordering just those updates?

- Perform loop through updates
- If update is not valid
    - Reorder update
        - Get broken rules
        - Perform swap for broken rules
- Add middle number to counter

In [18]:
class Rule:
    def __init__(self, value):
        self.value = value
        self.appears_before = rules_graph[value]

    def __lt__(self, other):
        return other.value in self.appears_before

    def __repr__(self):
        return str(self.value)

    def __int__(self):
        return int(self.value)


In [19]:
rules = [(2,3),(3,4)]
rules_graph = build_graph(rules)
rule_vectorized = np.vectorize(Rule)

In [20]:
rules_graph

defaultdict(set, {2: {3}, 3: {4}})

In [21]:
update = np.array([1, 4, 3, 2, 4])
sorted(rule_vectorized(update))

[1, 2, 3, 4, 4]

In [22]:
rules, updates = load_rules_and_updates(data)
rules_graph = build_graph(rules)

sum_corrected = 0
for update in updates:
    if not is_update_valid(update, rules, rules_graph):
        corrected_update = [int(u) for u in sorted(rule_vectorized(update))]
        sum_corrected += corrected_update[len(update)//2]
        
sum_corrected

5723